In [1]:
from bs4 import BeautifulSoup
import requests
import re

In [2]:
biura_nieruchomosci_url = "https://www.otodom.pl/warszawa/firmy/biura-nieruchomosci/?search%5Bregion_id%5D=7&search%5Bsubregion_id%5D=197&search%5Bcity_id%5D=26&sq=&page="
pages_range = range(1, 69)

In [7]:
response = requests.get(biura_nieruchomosci_url + "1")
soup = BeautifulSoup(response.content, 'html.parser')

In [9]:
biura = soup.find_all(class_="company-item")

In [10]:
tmp = biura[0]
tmp

<article class="company-item">
<figure class="company-item-image">
<a class="img-shadow" href="https://koloseum-nieruchomosci.otodom.pl" title="Deweloperzy: KOLOSEUM NIERUCHOMOŚCI - Warszawa, mazowieckie"><img alt="Deweloperzy: KOLOSEUM NIERUCHOMOŚCI - Warszawa, mazowieckie" src="https://ireland.apollo.olxcdn.com/v1/files/eyJmbiI6ImV6MjM2NGJ4MWdibDMtQVBMIn0.L-sslzPRC94fTqGrfWW7quvca59kpGhkM3MzwNDwN68/image;s=219x163;q=80"/></a>
</figure>
<div class="company-item-content">
<header>
<h4 class="company-item-title">
<a href="https://koloseum-nieruchomosci.otodom.pl" title="Deweloperzy: KOLOSEUM NIERUCHOMOŚCI - Warszawa, mazowieckie">
                                            KOLOSEUM NIERUCHOMOŚCI                                        </a>
</h4>
<span class="company-item-since">09.08.2012</span>
</header>
<p class="company-item-address">Domaniewska 37/31G<br/>02-672 Warszawa, mazowieckie</p>
<div class="company-item-properties">
<strong>sprzedaż:</strong> 3646 <span>od:</span> 25 000 zł

In [12]:
name = tmp.find(class_="company-item-title").get_text().strip()
print(name)
since = tmp.find(class_="company-item-since").get_text().strip()
print(since)
ulica, numer = tmp.find(class_="company-item-address").contents[0].split(" ", 1)
print(ulica)
print(numer)
part1, wojewodztwo = tmp.find(class_="company-item-address").contents[2].split(",")
wojewodztwo = wojewodztwo.strip()
kod, miasto = part1.replace(",", "").split(" ", 1)
print(wojewodztwo)
print(miasto)
print(kod)
url = tmp.find(class_="company-item-title").a["href"]
print(url)
url_tmp = requests.get(url)
url_soup = BeautifulSoup(url_tmp.content, 'html.parser')
about = url_soup.find(class_="shop-about").p.get_text()
print(about)
info_re = re.compile("sprzedaż: ([\d ]+)")# .*wynajem: ([\d ]+).*
info_tmp = tmp.find(class_="company-item-properties").get_text().strip()
print(info_tmp)
m = info_re.match(info_tmp)
m.groups()

KOLOSEUM NIERUCHOMOŚCI
09.08.2012
Domaniewska
37/31G
mazowieckie
Warszawa
02-672
https://koloseum-nieruchomosci.otodom.pl
Biuro nieruchomości Koloseum specjalizuje się w sprzedaży i wynajmie nieruchomości, działek, mieszkań, domów i lokali użytkowych na terenie Warszawy oraz innych regionów kraju. W naszej bazie znajdują się najciekawsze propozycje ofert. Dzięki wieloletniemu doświadczeniu naszych specjalistów, jesteśmy w stanie nie tylko zestawić kompleksowe i atrakcyjne oferty kupna, sprzedaży  wszelkiego rodzaju nieruchomości, ale również doradzić najlepszy wybór, mając na uwadze sugestie klientów.
sprzedaż: 3646 od: 25 000 zł do: 17 000 000 zł


('3646 ',)

In [16]:
import pandas as pd

data = []
for page in pages_range:
    response = requests.get(biura_nieruchomosci_url + "1")
    soup = BeautifulSoup(response.content, 'html.parser')
    biura = soup.find_all(class_="company-item")
    for biuro in biura:
        name = biuro.find(class_="company-item-title").get_text().strip()
        since = biuro.find(class_="company-item-since").get_text().strip()
        ulica = biuro.find(class_="company-item-address").contents[0]
        part1, wojewodztwo = biuro.find(class_="company-item-address").contents[2].split(",")
        wojewodztwo = wojewodztwo.strip()
        kod, miasto = part1.replace(",", "").split(" ", 1)
        url = biuro.find(class_="company-item-title").a["href"]
        data.append({"name": name, "since": since, "address": ulica, "province": wojewodztwo,
                     "postcode": kod, "city": miasto, "url": url})

data_df = pd.DataFrame(data)

In [17]:
data_df

,name,since,address,province,postcode,city,url
0,KOLOSEUM NIERUCHOMOŚCI,09.08.2012,Domaniewska 37/31G,mazowieckie,02-672,Warszawa,https://koloseum-nieruchomosci.otodom.pl
1,Freedom Nieruchomości Franczyza,17.04.2018,Pokorna 2 lok. U13,mazowieckie,00-199,Warszawa,https://www.otodom.pl/shop/freedom-nieruchomos...
2,Expander Advisors,30.04.2015,Ul. Inflancka 4b,mazowieckie,00-189,Warszawa,https://www.otodom.pl/shop/expander-advisors-I...
3,Biuro sprzedaży,02.11.2015,Marynarska 21 lok 6,mazowieckie,02-674,Warszawa,https://N-plus.otodom.pl
4,Metrohouse Franchise S.A.,02.07.2018,Żelazna 28/30,mazowieckie,00-832,Warszawa,https://metrohouse-franchise.otodom.pl
...,...,...,...,...,...,...,...
1355,Euro-House Nieruchomości komercyjne,04.02.2014,ul. Braci Wagów 20 lok u1,mazowieckie,02-791,Warszawa,https://www.otodom.pl/shop/euro-house-nierucho...
1356,Biuro Na Miarę,20.02.2018,Nowy Świat 47,mazowieckie,00-042,Warszawa,https://biuronamiare.otodom.pl
1357,Flatcare - nowoczesny zarządca nieruchomości,02.12.2019,Kępa Tarchomińska 6C/1/2,mazowieckie,03-113,Warszawa,https://flatcare.otodom.pl
1358,En Casa Premium Real Estate,19.11.2014,Studencka 55,mazowieckie,02-735,Warszawa,https://encasa.otodom.pl


In [18]:
data_df.to_csv('biura.csv')